In [1]:
import pandas as pd, numpy as np

# UCI Cleveland subset (14 features + target) -------------------------------
url = "https://raw.githubusercontent.com/sharmaroshan/Heart-UCI-Dataset/master/heart.csv"
df = pd.read_csv(url)              # 303 rows


In [2]:
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      1

3. Bias Identification 

In [7]:
# 1. Imports ──────────────────────────────────────────────────────────────
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from fairlearn.metrics import (
    MetricFrame, selection_rate,
    demographic_parity_difference, demographic_parity_ratio
)

# 2. Data load (your CSV already in `df`) ──────────────────────────────────
# df = pd.read_csv("heart.csv")     # if reading from file
X = df.drop('target', axis=1)       # label column is 'target'
y = df['target']
A = df['sex']                       # protected attribute (0 = female, 1 = male)

# 3. Train/test split & baseline model ────────────────────────────────────
X_tr, X_te, y_tr, y_te, A_tr, A_te = train_test_split(
    X, y, A, stratify=y, test_size=0.25, random_state=0
)

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=200, solver='lbfgs'))
]).fit(X_tr, y_tr)

pred = pipe.predict(X_te)

# 4. Grouped performance with MetricFrame ─────────────────────────────────
base_metrics = {                   # ONLY metrics that need y_true & y_pred
    'accuracy'      : accuracy_score,
    'selection_rate': selection_rate
}

mf = MetricFrame(
    metrics            = base_metrics,
    y_true             = y_te,
    y_pred             = pred,
    sensitive_features = A_te
)

print("Per-sex breakdown")
print(mf.by_group, "\n")
print("Overall performance")
print(mf.overall, "\n")

# 5. Global fairness metrics (need sensitive_features) ────────────────────
dp_diff  = demographic_parity_difference(y_te, pred, sensitive_features=A_te)
dp_ratio = demographic_parity_ratio(   y_te, pred, sensitive_features=A_te)

print(f"Demographic-parity difference : {dp_diff:.3f}")
print(f"Demographic-parity ratio      : {dp_ratio:.3f}")


Per-sex breakdown
     accuracy  selection_rate
sex                          
0    0.966667             0.8
1    0.804348             0.5 

Overall performance
accuracy          0.868421
selection_rate    0.618421
dtype: float64 

Demographic-parity difference : 0.300
Demographic-parity ratio      : 0.625


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy={0: A_train.value_counts()[1]}, random_state=0)  # balance sex
X_bal, y_bal = sm.fit_resample(X_train, y_train)
A_bal = sm.fit_resample(A_train.values.reshape(-1,1), y_train)[0].ravel()

In [8]:
# 1. Imports ──────────────────────────────────────────────────────────────
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from fairlearn.metrics import (
    MetricFrame, selection_rate,
    demographic_parity_difference, demographic_parity_ratio
)

# 2. Data load (your CSV already in `df`) ──────────────────────────────────
# df = pd.read_csv("heart.csv")     # if reading from file
X = df.drop('target', axis=1)       # label column is 'target'
y = df['target']
A = df['sex']                       # protected attribute (0 = female, 1 = male)

# 3. Train/test split & baseline model ────────────────────────────────────
X_tr, X_te, y_tr, y_te, A_tr, A_te = train_test_split(
    X, y, A, stratify=y, test_size=0.25, random_state=0
)

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=200, solver='lbfgs'))
]).fit(X_tr, y_tr)

pred = pipe.predict(X_te)

# 4. Grouped performance with MetricFrame ─────────────────────────────────
base_metrics = {                   # ONLY metrics that need y_true & y_pred
    'accuracy'      : accuracy_score,
    'selection_rate': selection_rate
}

mf = MetricFrame(
    metrics            = base_metrics,
    y_true             = y_te,
    y_pred             = pred,
    sensitive_features = A_te
)

print("Per-sex breakdown")
print(mf.by_group, "\n")
print("Overall performance")
print(mf.overall, "\n")

# 5. Global fairness metrics (need sensitive_features) ────────────────────
dp_diff  = demographic_parity_difference(y_te, pred, sensitive_features=A_te)
dp_ratio = demographic_parity_ratio(   y_te, pred, sensitive_features=A_te)

print(f"Demographic-parity difference : {dp_diff:.3f}")
print(f"Demographic-parity ratio      : {dp_ratio:.3f}")


Per-sex breakdown
     accuracy  selection_rate
sex                          
0    0.966667             0.8
1    0.804348             0.5 

Overall performance
accuracy          0.868421
selection_rate    0.618421
dtype: float64 

Demographic-parity difference : 0.300
Demographic-parity ratio      : 0.625
